In [1]:
import torch
from transformer import *

In [2]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device = get_device()

In [9]:
import torch
from transformer import Transformer

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

trg_vocab = sorted(list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>')))
src_vocab = sorted(list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>')))

trg_vocab.append(START_TOKEN)
trg_vocab.append(END_TOKEN)
trg_vocab.append(PADDING_TOKEN)

src_vocab.append(START_TOKEN)
src_vocab.append(END_TOKEN)
src_vocab.append(PADDING_TOKEN)

index_to_trg = {k:v for k,v in enumerate(trg_vocab)}
trg_to_index = {v:k for k,v in enumerate(trg_vocab)}
index_to_src = {k:v for k,v in enumerate(src_vocab)}
src_to_index = {v:k for k,v in enumerate(src_vocab)}

d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.2
num_layers = 1
max_sequence_length = 200
trg_vocab_size = len(trg_vocab)

ru_eng = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)
eng_ru = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)

ru_eng.load_state_dict(torch.load("models/ru_eng_trans.pth"))
eng_ru.load_state_dict(torch.load("models/eng_ru_trans.pt"))

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device = get_device()

ru_eng.to(device)
eng_ru.to(device)

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(152, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (ln): LayerNorm((512,), eps=512, elementwise_affine=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (ffn): FeedFoward(
          (net): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.2, inplace=False)
          )
        )
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout2): Dropout(p=0.2, inplace=F

In [10]:
ru_eng.translate("Я люблю группу король и шут")

'- Executive Board States'

In [11]:
eng_ru.translate("Good night")

'Проект дня'